### Hierarchical Planning Example: Family Vacation to Italy

Scenario Overview: Planning an international vacation involves many interconnected decisions such as budgeting, travel arrangements, accommodation, local transport, and activity selection. Without structured planning, the process becomes overwhelming and time-consuming.

Approach: This use case demonstrates Hierarchical Planning, where a high-level goal is broken down into organized sub-tasks and executable steps. The AI agent analyzes the final objective—“Plan a family vacation to Italy”—and transforms it into a multi-layer plan:

🔹 Level 1: High-Level Goal

→ Plan a complete 7-day family trip to Italy.

🔹 Level 2: Sub-Goals

Set travel budget

Book flights

Reserve accommodations

Plan local travel

Create sightseeing/activity schedule

Organize food + dietary considerations

🔹 Level 3: Actionable Steps

Example breakdown:

| Sub-goal        | Action Steps                                                                                           |
| --------------- | ------------------------------------------------------------------------------------------------------ |
| Budgeting       | Estimate flight cost, hotel pricing, activities, food allowance                                        |
| Flights         | Compare airlines, choose departure dates, book tickets                                                 |
| Accommodation   | Evaluate Airbnb vs hotels, check proximity to landmarks, confirm reservation                           |
| Activities      | Create daily itinerary (Rome → Florence → Venice), book museum passes, identify child-friendly options |
| Local Transport | Lookup rail passes, taxi availability, or rental car rules                                             |
| Food Planning   | Identify restaurants, check dietary needs, make reservations                                           |



Outcome

The result is a structured travel plan that transforms a vague intention into clear, prioritized execution steps.

✔ Reduced planning stress
✔ Efficient cost and time management
✔ Smart task sequencing
✔ Clear blueprint for execution

In [ ]:
# !pip install crewai==0.28.8 crewai_tools==0.1.6


In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')
import os
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
# Load environment variables
load_dotenv()

import warnings
warnings.filterwarnings('ignore')
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)


In [2]:
# CrewAI: Hierarchical Planning — Family Vacation to Italy
# Agents: Planner, Researcher, Budgeter, ItineraryMaker, Coordinator
# Dynamic inputs: vacation_goal, dates, family_size, preferences

from crewai import Agent, Task, Crew

# ----------------------
# Agents
# ----------------------

planner = Agent(
    role="High-Level Trip Planner",
    goal="Decompose the vacation goal into a hierarchy of sub-goals and prioritized tasks.",
    backstory="You break down large travel goals into clear, ordered subtasks and define success criteria.",
    llm=llm,
    verbose=True
)

researcher = Agent(
    role="Travel Researcher",
    goal="Find practical options for flights, accommodations, and local transport given constraints.",
    backstory="You are skilled at quickly comparing travel options and returning concise recommendations.",
    llm=llm,
    verbose=True
)

budgeter = Agent(
    role="Travel Budget Analyst",
    goal="Estimate costs across flights, hotels, transport, food, and activities and recommend a budget.",
    backstory="You create realistic cost estimates and suggest cost-saving tradeoffs for families.",
    llm=llm,
    verbose=True
)

itinerary_maker = Agent(
    role="Itinerary Creator",
    goal="Produce a detailed day-by-day itinerary aligned with family preferences, pace, and child-friendly options.",
    backstory="You craft balanced itineraries with travel time, rest, and high-impact experiences.",
    llm=llm,
    verbose=True
)

coordinator = Agent(
    role="Trip Coordinator",
    goal="Assemble outputs from other agents into a single cohesive travel plan and checklist.",
    backstory="You synthesize plans into an actionable final deliverable and list next steps for booking.",
    llm=llm,
    verbose=True
)


In [3]:
# Tasks (use dynamic placeholders)

decompose_task = Task(
    description=(
        "Decompose the high-level goal:\n"
        "Goal: {vacation_goal}\n"
        "Dates: {dates}\n"
        "Family size: {family_size}\n"
        "Preferences: {preferences}\n\n"
        "Produce a hierarchical plan with levels:\n"
        "Level 1: Major pillars (travel, lodging, transport, activities, budget)\n"
        "Level 2: For each pillar, list 3-5 concrete sub-tasks\n"
        "Level 3: For the top-priority subtasks, suggest success criteria and timelines.\n\n"
        "Return as a structured list or JSON-like text."
    ),
    expected_output="Hierarchical plan (pillars -> subtasks -> success criteria)",
    agent=planner
)

research_task = Task(
    description=(
        "Research travel options given:\n"
        "Goal: {vacation_goal}\n"
        "Dates: {dates}\n"
        "Family size: {family_size}\n"
        "Preferences: {preferences}\n\n"
        "Return concise recommendations for:\n"
        "- 2 flight options (price tier and rationale)\n"
        "- 2 accommodation types (hotel vs apartment) and one sample neighborhood each\n"
        "- Local transport options (rail passes, car rental notes)\n\n"
        "If any option requires online bookings or external checks, flag it with 'ACTION REQUIRED'."
    ),
    expected_output="Research summary with 2-3 recommended options per category",
    agent=researcher
)

budget_task = Task(
    description=(
        "Create a budget estimate for the trip described:\n"
        "Goal: {vacation_goal}\n"
        "Dates: {dates}\n"
        "Family size: {family_size}\n"
        "Preferences: {preferences}\n\n"
        "Include estimated costs (per category): Flights, Accommodation (per night), Local transport, Food (per day), Activities, Contingency.\n"
        "Give a 'Total estimated budget' and two cost-optimization suggestions."
    ),
    expected_output="Budget table and cost-saving recommendations",
    agent=budgeter
)

itinerary_task = Task(
    description=(
        "Build a 7-day sample itinerary (adjust to {dates} length if needed) for the family:\n"
        "Goal: {vacation_goal}\n"
        "Family size: {family_size}\n"
        "Preferences: {preferences}\n\n"
        "For each day list:\n"
        "- Morning / Afternoon / Evening items\n"
        "- Estimated travel time between stops\n"
        "- Child-friendly notes and booking needs\n"
        "- Time buffers for rest\n\n"
        "Return a balanced, realistic daily schedule."
    ),
    expected_output="Day-by-day itinerary with notes",
    agent=itinerary_maker
)

coordinate_task = Task(
    description=(
        "Assemble final travel plan using previous outputs.\n"
        "Inputs: hierarchical plan, research summary, budget estimate, itinerary.\n\n"
        "Produce a final deliverable containing:\n"
        "- Executive summary (1-2 paragraphs)\n"
        "- Actionable booking checklist (flights, hotels, passes, reservations)\n"
        "- Timeline for when to book each item\n"
        "- Risk notes and contingencies\n"
        "- Final total estimated cost\n\n"
        "Format clearly for sharing with family."
    ),
    expected_output="Final trip plan & checklist",
    agent=coordinator
)

In [4]:
# Build Crew

vacation_crew = Crew(
    llm=llm,
    agents=[planner, researcher, budgeter, itinerary_maker, coordinator],
    tasks=[decompose_task, research_task, budget_task, itinerary_task, coordinate_task],
    verbose=2
)

In [5]:
vacation_goal = "Plan a 7-day family vacation to Italy focusing on Rome, Florence, and Venice."
dates = "2025-06-10 to 2025-06-17"
family_size = "2 adults, 2 children (ages 8 and 5)"
preferences = "Moderate budget, kid-friendly activities, avoid long overnight travel, prefer central accommodations"


In [ ]:
inputs = {
        "vacation_goal": vacation_goal,
        "dates": dates,
        "family_size": family_size,
        "preferences": preferences
    }

result = vacation_crew.kickoff(inputs)
print("\n--- FINAL VACATION PLAN ---\n")
print(result)

 [DEBUG]: == Working Agent: High-Level Trip Planner
 [INFO]: == Starting Task: Decompose the high-level goal:
Goal: Plan a 7-day family vacation to Italy focusing on Rome, Florence, and Venice.
Dates: 2025-06-10 to 2025-06-17
Family size: 2 adults, 2 children (ages 8 and 5)
Preferences: Moderate budget, kid-friendly activities, avoid long overnight travel, prefer central accommodations

Produce a hierarchical plan with levels:
Level 1: Major pillars (travel, lodging, transport, activities, budget)
Level 2: For each pillar, list 3-5 concrete sub-tasks
Level 3: For the top-priority subtasks, suggest success criteria and timelines.

Return as a structured list or JSON-like text.


> Entering new CrewAgentExecutor chain...
Thought: I need to break down the vacation goal into a structured hierarchical plan, focusing on the major pillars and their respective subtasks, along with success criteria and timelines.

Action: Delegate work to co-worker
Action Input: {
    "coworker": "Travel Resear